In [108]:
import os
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Đọc dữ liệu

In [109]:
def read_parquet_by_type(train_path: str):
    # Lấy tất cả các file parquet trong thư mục
    files = [os.path.join(train_path, f) for f in os.listdir(train_path) if f.endswith('.parquet')]
    
    # Phân loại các file theo loại tên
    user_chunk_files = [file for file in files if 'user_chunk' in file]
    purchase_history_chunk_files = [file for file in files if 'purchase_history_daily_chunk' in file]
    item_chunk_files = [file for file in files if 'item_chunk' in file]
    
    # Đọc các file riêng biệt thành DataFrame
    user_chunk_df = pl.concat([pl.read_parquet(file) for file in user_chunk_files]) if user_chunk_files else None
    purchase_history_chunk_df = pl.concat([pl.read_parquet(file) for file in purchase_history_chunk_files]) if purchase_history_chunk_files else None
    item_chunk_df = pl.concat([pl.read_parquet(file) for file in item_chunk_files]) if item_chunk_files else None
    
    # Trả về một dictionary chứa các DataFrame
    return {
        "user_chunk": user_chunk_df,
        "purchase_history_chunk": purchase_history_chunk_df,
        "item_chunk": item_chunk_df
    }

In [110]:
train_path = 'D:/2025-2026/HK1-2025-2026/CS116_Python_for_ML/recommendation_dataset'
dataframes = read_parquet_by_type(train_path)

In [111]:
df_user = dataframes["user_chunk"]
df_purchase = dataframes["purchase_history_chunk"]
df_item = dataframes["item_chunk"]

# Thêm Đặc Trưng Mới (Feature Engineering)

Trong bài toán dự đoán mua hàng, việc tạo ra các đặc trưng mới (Feature Engineering) từ dữ liệu thô đóng vai trò then chốt, quyết định phần lớn đến hiệu quả của mô hình. Dữ liệu ban đầu, đặc biệt là các cột mô tả sản phẩm như description, category, category_l1, l2, và l3, chứa đựng thông tin vô cùng giá trị nhưng lại ở dạng văn bản phi cấu trúc, khiến mô hình học máy khó có thể khai thác trực tiếp.

Nhận thấy rằng hành vi mua sắm của khách hàng thường phản ánh một nhu cầu hoặc một giai đoạn cụ thể trong cuộc sống (ví dụ: đang chuẩn bị sinh, có con trong độ tuổi ăn dặm...), nhóm đã quyết định xây dựng một bộ đặc trưng mới nhằm "phiên dịch" thông tin sản phẩm thành các nhóm nhu cầu của khách hàng.

Phương pháp thực hiện của chúng tôi dựa trên kỹ thuật trích xuất từ khóa (keyword extraction). Quy trình bao gồm việc phân tích sâu vào ngữ nghĩa của các cột mô tả để định nghĩa các nhóm khách hàng tiêu biểu (ví dụ: "Mẹ Bầu & Sau Sinh", "Phụ huynh Sơ sinh", "Người mua sắm cho Gia đình"...). Tương ứng với mỗi nhóm, một bộ từ khóa (keyword dictionary) đặc trưng đã được xây dựng. Bằng cách quét qua dữ liệu mô tả của từng sản phẩm, chúng tôi tiến hành tìm kiếm và gán sản phẩm đó vào nhóm khách hàng phù hợp nhất dựa trên sự hiện diện của các từ khóa này.

Kết quả của quá trình này là một đặc trưng phân loại (categorical feature) mới, giúp mô hình không chỉ "nhìn" thấy một sản phẩm là "sữa bột" mà còn "hiểu" rằng sản phẩm này phục vụ cho nhóm "Phụ huynh có con Sơ sinh". Đặc trưng này hứa hẹn sẽ làm tăng đáng kể độ chính xác và khả năng diễn giải cho mô hình dự đoán của chúng tôi.

## In ra các giá trị unique trong các cột

In [112]:
#print(df_item['category'].unique().to_list())

In [113]:
#print(df_item['category_l1'].unique().to_list())

In [114]:
#print(df_item['category_l2'].unique().to_list())

In [115]:
#print(df_item['category_l3'].unique().to_list())

## Xử lí các giá trị không xác định

In [116]:
# Định nghĩa các từ khóa tìm kiếm cho gender_target
object_target_keywords = {
    # --- CÁC NHÓM THEO ĐỐI TƯỢNG SỬ DỤNG ---
    "Bé Gái": [
        'bé gái', 'girl', 'đầm', 'váy', 'cài', 'kẹp', 'cột'
    ],
    "Bé Trai": [
        'bé trai', 'boy'
    ],
    "Sơ sinh": [
        'sơ sinh', 'newborn', 'ss', '0-1y'
    ],
    "Unisex": [
        'unisex', 'tã', 'bỉm', 'bình sữa', 'núm ty', 'xe đẩy', 'địu'
    ],
    "Nữ": [
        'cho mẹ', 'mẹ bầu', 'sau sinh', 'băng vệ sinh', 'bvs', 'dưỡng da', 'collagen', 'lợi sữa'
    ],
    "Nam": [
        'nam giới', 'men', 'for men'
    ],
    "Người lớn": [
        'gia đình', 'người lớn', 'cho cả nhà', 'khẩu trang người lớn', 'giặt xả', 'nước lau sàn'
    ],

    # --- CÁC NHÓM THEO NHU CẦU/MỤC ĐÍCH SỬ DỤDNG ---
    "Dinh dưỡng & Cho ăn": [
        'sữa', 'sữa bột', 'sữa tươi', 'sữa chua', 'váng sữa', 'sữa non', 'sữa hạt', 'sữa đặc trị',
        'ăn dặm', 'bột ăn dặm', 'cháo ăn dặm', 'cháo tươi', 'bánh gạo', 'snack ăn dặm',
        'trái cây nghiền', 'rau củ nghiền', 'hạt nêm', 'dashi', 'ghế ngồi ăn', 'yếm'
    ],
    "Sức khỏe & An toàn": [
        'tpcn', 'vitamin', 'dha', 'omega-3', 'canxi', 'sắt', 'kẽm',
        'men vi sinh', 'men tiêu hóa', 'tăng cường sức đề kháng',
        'nước muối sinh lý', 'hút mũi', 'gạc', 'rơ lưỡi', 'kem chống hăm', 'dầu tràm', 'hạ sốt'
    ],
    "Đồ chơi & Phát triển": [
        'đồ chơi', 'gặm nướu', 'xúc xắc', 'đồ chơi nước', 'đồ chơi nhồi bông',
        'phát triển giác quan', 'học tập và phát triển tư duy',
        'vận động', 'xe tập đi', 'thảm chơi', 'kệ chữ a', 'sách'
    ]
}

In [117]:
# Trước khi xử lí 
print(df_item['gender_target'].value_counts())

shape: (5, 2)
┌────────────────┬───────┐
│ gender_target  ┆ count │
│ ---            ┆ ---   │
│ str            ┆ u32   │
╞════════════════╪═══════╡
│ Unisex         ┆ 6     │
│ Bé Trai        ┆ 3318  │
│ Không xác định ┆ 18038 │
│ Sơ sinh        ┆ 1862  │
│ Bé Gái         ┆ 4108  │
└────────────────┴───────┘


In [118]:
import polars as pl


def fill_gender_target_legacy_6_cols(df: pl.DataFrame) -> pl.DataFrame:
    """
    Điền giá trị cho 'gender_target' bằng cách duyệt qua 6 cột,
    vẫn giữ cấu trúc vòng lặp for và cập nhật df tuần tự.
    """
    condition_is_unknown = pl.col('gender_target') == 'Không xác định'

    for gender, keywords in object_target_keywords.items():
        regex_pattern = '|'.join(keywords)
        cond_des = pl.col('description').str.contains(regex_pattern)
        cond_des_new = pl.col('description_new').str.contains(regex_pattern)
        cond_category = pl.col('category').str.contains(regex_pattern)
        cond_category_l1 = pl.col('category_l1').str.contains(regex_pattern)
        cond_l2 = pl.col('category_l2').str.contains(regex_pattern)
        cond_l3 = pl.col('category_l3').str.contains(regex_pattern)

        df = df.with_columns(
            pl.when(condition_is_unknown)
            .then(
                # ===== PHẦN SỬA LỖI BẮT ĐẦU TỪ ĐÂY =====
                pl.when(cond_des).then(pl.lit(gender))
                .otherwise(
                    pl.when(cond_des_new).then(pl.lit(gender))
                    .otherwise(
                        pl.when(cond_category).then(pl.lit(gender))
                        .otherwise(
                            pl.when(cond_category_l1).then(pl.lit(gender))
                            .otherwise(
                                pl.when(cond_l2).then(pl.lit(gender))
                                .otherwise(
                                    pl.when(cond_l3).then(pl.lit(gender))
                                    .otherwise(pl.lit("Không xác định"))
                                )
                            )
                        )
                    )
                )
                # ===== KẾT THÚC PHẦN SỬA LỖI =====
            )
            .otherwise(pl.col('gender_target'))
            .alias('gender_target')
        )
    
    return df



# Áp dụng hàm
df_item = fill_gender_target_legacy_6_cols(df_item)
print(df_item)

shape: (27_332, 34)
┌────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ p_id   ┆ item_id    ┆ price      ┆ category_ ┆ … ┆ volume    ┆ material  ┆ sale_stat ┆ descripti │
│ ---    ┆ ---        ┆ ---        ┆ l1_id     ┆   ┆ ---       ┆ ---       ┆ us        ┆ on_new    │
│ i32    ┆ str        ┆ decimal[38 ┆ ---       ┆   ┆ str       ┆ str       ┆ ---       ┆ ---       │
│        ┆            ┆ ,4]        ┆ i32       ┆   ┆           ┆           ┆ i32       ┆ str       │
╞════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 17065  ┆ 0502020000 ┆ 99000.0000 ┆ 1         ┆ … ┆ Không xác ┆ Không xác ┆ 0         ┆ Chi tiết  │
│        ┆ 004        ┆            ┆           ┆   ┆ định      ┆ định      ┆           ┆ sản phẩm  │
│        ┆            ┆            ┆           ┆   ┆           ┆           ┆           ┆ …         │
│ 72370  ┆ 0010290040 ┆ 69000.0000 ┆ 3292      ┆ … ┆ Không xác ┆ Không 

In [119]:
df_item = df_item.rename({'gender_target': 'object_target'})
print(df_item['object_target'].value_counts())

shape: (11, 2)
┌──────────────────────┬───────┐
│ object_target        ┆ count │
│ ---                  ┆ ---   │
│ str                  ┆ u32   │
╞══════════════════════╪═══════╡
│ Nam                  ┆ 334   │
│ Unisex               ┆ 1145  │
│ Dinh dưỡng & Cho ăn  ┆ 861   │
│ Nữ                   ┆ 391   │
│ Sơ sinh              ┆ 5599  │
│ …                    ┆ …     │
│ Sức khỏe & An toàn   ┆ 144   │
│ Người lớn            ┆ 2703  │
│ Không xác định       ┆ 3739  │
│ Đồ chơi & Phát triển ┆ 612   │
│ Bé Trai              ┆ 4217  │
└──────────────────────┴───────┘


In [122]:
# Chỉ lấy các cột nên giữ lại ở task4
df_item = df_item.select([
    pl.col("p_id"), 
    pl.col("category"),
    pl.col("category_l1"),
    pl.col("created_date"),
    pl.col("creation_timestamp"),
    pl.col("price"),
    pl.col("gp"),
    pl.col("object_target"),
    pl.col("sale_status")
])

df_item.head()

p_id,category,category_l1,created_date,creation_timestamp,price,gp,object_target,sale_status
i32,str,str,datetime[μs],i64,"decimal[38,4]","decimal[38,4]",str,i32
17065,"""Núm ty Dr Brown""","""Babycare""",2012-04-04 09:25:44.240,1333531544,99000.0000,36828.0000,"""Unisex""",0
72370,"""Bộ quần áo bé gái""","""Thời trang""",2017-08-18 08:50:50.713,1503046250,69000.0000,0.0000,"""Bé Gái""",0
31154,"""Gặm nướu khác""","""Đồ chơi & Sách""",2013-01-18 09:33:04.260,1358501584,45000.0000,14490.0000,"""Đồ chơi & Phát triển""",0
46123,"""Merries_Sơ Sinh""","""Tã""",2014-05-14 10:07:19.603,1400062039,401000.0000,59749.0000,"""Unisex""",0
46127,"""Merries_Tã Quần""","""Tã""",2014-05-14 10:07:20.370,1400062040,401000.0000,65764.0000,"""Unisex""",0
